<h2 style="color:green" align="center">Efficient Data Layout and retrieval strategy with reduced storage cost</2>

### Importing the required packages

In [20]:
import time
import psycopg2
import pandas as pd

### Checking the connection with PostGreSQL

In [21]:
start_time = time.time()
conn = psycopg2.connect(
    host="localhost",
    database="coursera",
    user="postgres",
    password="rocky1405!")
print("execution_time: ", time.time() - start_time)

execution_time:  0.008376121520996094


### Functions to hit the DB & get the results

In [23]:
def run_query(command):
    conn = psycopg2.connect(
            host="localhost",
            database="coursera",
            user="postgres",
            port=5432,
            password="rocky1405!")
    cur = conn.cursor()
    cur.execute(command)
    cur.close()
    conn.commit()
    conn.close()

def run_queries(command):
    conn = psycopg2.connect(
            host="localhost",
            database="coursera",
            port=5432,
            user="postgres",
            password="rocky1405!")
    cur = conn.cursor()
    cur.execute(command)
    rows = cur.fetchall()
    cur.close()
    conn.commit()
    conn.close()
    return rows

### Loading the data

In [18]:
df = pd.read_csv("Coursera-1.csv")
df.head()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


### Modifying the Data

Course Rating mostly has numbers ranging from 0+ to 5. However for new courses or courses where  participants are yet to use, the rating is not present. It is marked as "Not Calibrated". In the user interface that would show up as having 0 ratings. So modifying "Not Calibrated" to "0".

In [34]:
df["Course Rating"] = df["Course Rating"].replace(to_replace = ["Not Calibrated"], value = [0])

### Creating the table to dump the data as is

In [38]:
def create_master_table():
    """ create tables in the PostgreSQL database"""
    command = """
            create table master_data (
                course_name VARCHAR, 
                university VARCHAR, 
                difficulty_level VARCHAR, 
                course_rating FLOAT, 
                course_url VARCHAR, 
                course_description VARCHAR, 
                skills VARCHAR
                )
        """
    run_query(command)
create_master_table()

### Inserting the data into master table

In [43]:
conn = psycopg2.connect(
    host="localhost",
    database="coursera",
    port=5432,
    user="postgres",
    password="rocky1405!")
conn.autocommit = True
cursor = conn.cursor()
for row in df.iterrows():
    cursor.execute("INSERT INTO master_data(course_name, university, difficulty_level, course_rating, course_url, course_description, skills) VALUES(%s, %s, %s, %s, %s, %s, %s)", (row[1]["Course Name"], row[1]["University"], row[1]["Difficulty Level"], row[1]["Course Rating"], row[1]["Course URL"], row[1]["Course Description"], row[1]["Skills"]))

### Checking the count to verify that all the data is loaded

In [44]:
command = """SELECT count(*) from master_data"""
number_of_rows = run_queries(command)
print(number_of_rows[0][0])

3522


### Checking for duplicates

If a row is present more than 1 time, the duplicates are removed & size column represents the number of times duplicate was present. 1 means no duplicate, 2 means presence of 1 duplicate & so on.

In [45]:
df = df.groupby(df.columns.tolist(),as_index=False).size()
df.head()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills,size
0,"""Making"" Progress Teach-Out",Emory University,Beginner,4.2,https://www.coursera.org/learn/makingprogress,What is �progress�? How do we decide when we�r...,education circle time Newspaper family hop...,1
1,(Non) Status quo Attitudes with NetLogo,Coursera Project Network,Conversant,0,https://www.coursera.org/learn/nonstatusquo-at...,Have you ever wondered what is behind differen...,constant (computer programming) interact eul...,1
2,1E Test: How Computers Work,Coursera,Advanced,0,https://www.coursera.org/learn/how-computers-w...,Test,physical address computer data storage name ...,1
3,3-Axis Machining with Autodesk Fusion 360,Autodesk,Intermediate,4.9,https://www.coursera.org/learn/3-axis-machinin...,As our machining geometry gets more complicate...,Mechanical Engineering representational state...,1
4,3D Art and Audio Pipeline,Unity,Advanced,4.3,https://www.coursera.org/learn/3d-art-and-audi...,"Welcome to 3D Art and Audio Pipeline, the thir...",Software Engineering software development pro...,1


### Adding record_id

In [46]:
df['record_id'] = list(range(1, df.shape[0] + 1))
df.head()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills,size,record_id
0,"""Making"" Progress Teach-Out",Emory University,Beginner,4.2,https://www.coursera.org/learn/makingprogress,What is �progress�? How do we decide when we�r...,education circle time Newspaper family hop...,1,1
1,(Non) Status quo Attitudes with NetLogo,Coursera Project Network,Conversant,0,https://www.coursera.org/learn/nonstatusquo-at...,Have you ever wondered what is behind differen...,constant (computer programming) interact eul...,1,2
2,1E Test: How Computers Work,Coursera,Advanced,0,https://www.coursera.org/learn/how-computers-w...,Test,physical address computer data storage name ...,1,3
3,3-Axis Machining with Autodesk Fusion 360,Autodesk,Intermediate,4.9,https://www.coursera.org/learn/3-axis-machinin...,As our machining geometry gets more complicate...,Mechanical Engineering representational state...,1,4
4,3D Art and Audio Pipeline,Unity,Advanced,4.3,https://www.coursera.org/learn/3d-art-and-audi...,"Welcome to 3D Art and Audio Pipeline, the thir...",Software Engineering software development pro...,1,5


### Creating a master table with distributed system

The master table would be partitioned by the column "size" which reflects the number of times a row is present in the actual data table. There are 4 tables into which the master table is partitioned based on the duplication factor.

<b>coursera_node_1:</b> will have data which has come only once in the data<br>
<b>coursera_node_2:</b> will have data which is present twice in the data<br>
<b>coursera_node_4:</b> will have the data occuring 3 or 4 times in the data<br>
<b>coursera_node_8:</b> will have the data occuring 4 to 8 times in the data<br>

Each of the node table is then partitioned into 4 based on the rating received.

<b>low_rating_node_1:</b> will have data where courses have ratings between 0 & 2 in the rows which have occured only 1 in the master data.<br>
<b>medium_rating_node_1:</b> will have data where courses have ratings between 2 & 4 in the rows which have occured only 1 in the master data.<br>
<b>good_rating_node_1:</b> will have data where courses have ratings between 4 & 4.51 in the rows which have occured only 1 in the master data.<br>
<b>great_rating_node_1:</b> will have data where courses have ratings between 4.51 & 5.1 in the rows which have occured only 1 in the master data.<br>

<i><b>Note:</b> values taken by partitioning dont take into consideration the upper limit. Hence if limit is mentioned as 5, 5 wont be considered in the partitioning. Hence, upper limit is mentioned as .1 more than the intended limit.</i>

In [85]:
def create_distributed_master_table():
    """ create tables in the PostgreSQL database"""
    command = """
            create table coursera_master (
                course_name VARCHAR,
                university VARCHAR,
                difficulty_level VARCHAR,
                course_rating FLOAT,
                course_url VARCHAR,
                course_description VARCHAR,
                skills VARCHAR,
                record_id NUMERIC,
                size NUMERIC
            ) PARTITION BY RANGE (size)
        """
    run_query(command)
create_distributed_master_table()

### Creating node for records occuring once

In [86]:
def create_node_1():
    """ create tables in the PostgreSQL database"""
    command_1 = """create table coursera_node_1 PARTITION OF coursera_master FOR VALUES FROM (1) to (1.1) PARTITION BY RANGE (course_rating)"""
    run_query(command_1)
    command_2 = """create table low_rating_node_1 PARTITION OF coursera_node_1 FOR VALUES FROM (0) TO (2)"""
    run_query(command_2)
    command_3 = """create table medium_rating_node_1 PARTITION OF coursera_node_1 FOR VALUES FROM (2) TO (4)"""
    run_query(command_3)
    command_4 = """create table good_rating_node_1 PARTITION OF coursera_node_1 FOR VALUES FROM (4) TO (4.51)"""
    run_query(command_4)
    command_5 = """create table great_rating_node_1 PARTITION OF coursera_node_1 FOR VALUES FROM (4.51) TO (5.1)"""
    run_query(command_5)
create_node_1()

### Creating node for records occuring twice

In [87]:
def create_node_2():
    """ create tables in the PostgreSQL database"""
    command_1 = """create table coursera_node_2 PARTITION OF coursera_master FOR VALUES FROM (2) to (2.1) PARTITION BY RANGE (course_rating)"""
    run_query(command_1)
    command_2 = """create table low_rating_node_2 PARTITION OF coursera_node_2 FOR VALUES FROM (0) TO (2)"""
    run_query(command_2)
    command_3 = """create table medium_rating_node_2 PARTITION OF coursera_node_2 FOR VALUES FROM (2) TO (4)"""
    run_query(command_3)
    command_4 = """create table good_rating_node_2 PARTITION OF coursera_node_2 FOR VALUES FROM (4) TO (4.51)"""
    run_query(command_4)
    command_5 = """create table great_rating_node_2 PARTITION OF coursera_node_2 FOR VALUES FROM (4.51) TO (5.1)"""
    run_query(command_5)
create_node_2()

### Creating node for records occuring 3 or 4 times

In [88]:
def create_node_4():
    """ create tables in the PostgreSQL database"""
    command_1 = """create table coursera_node_4 PARTITION OF coursera_master FOR VALUES FROM (3) to (4.1) PARTITION BY RANGE (course_rating)"""
    run_query(command_1)
    command_2 = """create table low_rating_node_4 PARTITION OF coursera_node_4 FOR VALUES FROM (0) TO (2)"""
    run_query(command_2)
    command_3 = """create table medium_rating_node_4 PARTITION OF coursera_node_4 FOR VALUES FROM (2) TO (4)"""
    run_query(command_3)
    command_4 = """create table good_rating_node_4 PARTITION OF coursera_node_4 FOR VALUES FROM (4) TO (4.51)"""
    run_query(command_4)
    command_5 = """create table great_rating_node_4 PARTITION OF coursera_node_4 FOR VALUES FROM (4.51) TO (5.1)"""
    run_query(command_5)
create_node_4()

### Creating node for records occuring between 4 to 8 times

In [89]:
def create_node_8():
    """ create tables in the PostgreSQL database"""
    command_1 = """create table coursera_node_8 PARTITION OF coursera_master FOR VALUES FROM (4.1) to (8.1) PARTITION BY RANGE (course_rating)"""
    run_query(command_1)
    command_2 = """create table low_rating_node_8 PARTITION OF coursera_node_8 FOR VALUES FROM (0) TO (2)"""
    run_query(command_2)
    command_3 = """create table medium_rating_node_8 PARTITION OF coursera_node_8 FOR VALUES FROM (2) TO (4)"""
    run_query(command_3)
    command_4 = """create table good_rating_node_8 PARTITION OF coursera_node_8 FOR VALUES FROM (4) TO (4.51)"""
    run_query(command_4)
    command_5 = """create table great_rating_node_8 PARTITION OF coursera_node_8 FOR VALUES FROM (4.51) TO (5.1)"""
    run_query(command_5)
create_node_8()

### Inserting data into the master table with partitioning

In [90]:
conn = psycopg2.connect(
    host="localhost",
    database="coursera",
    port=5432,
    user="postgres",
    password="rocky1405!")
conn.autocommit = True
cursor = conn.cursor()
for row in df.iterrows():
    cursor.execute("INSERT INTO coursera_master(course_name, university, difficulty_level, course_rating, course_url, course_description, skills, record_id, size) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)", (row[1]["Course Name"], row[1]["University"], row[1]["Difficulty Level"], row[1]["Course Rating"], row[1]["Course URL"], row[1]["Course Description"], row[1]["Skills"], row[1]["record_id"], row[1]["size"]))

### Cost Analysis

The query immediately below is run on the master table with no distributed system & partitioning. The cost to run the query is 644.83ms.

The cell below that has the query run on the master table with distributed system & partitioning. The cost to run the query is 228.07ms. 

On comparison, the cost to run the same query on table without partitioning was almost 3 times the when compared to the cost of the query run on table with partitioning. This gives the us an idea of how efficient the distributed architecture is.

On further analysis of the explanation of query on partitioned system, we can see that since rating was asked between 4.1 & 4.3 the search happened only in good_rating_nodes as compared to search on all the table. This helps in achieving the required efficiency.

In [96]:
command = """EXPLAIN select course_name, university, course_rating from master_data where course_rating >= 4.1 AND course_rating <= 4.3;"""
explanation = run_queries(command)
for row in explanation:
    print(row[0])

Seq Scan on master_data  (cost=0.00..644.83 rows=371 width=77)
  Filter: ((course_rating >= '4.1'::double precision) AND (course_rating <= '4.3'::double precision))


In [95]:
command = """EXPLAIN select course_name, university, course_rating from coursera_master where course_rating >= 4.1 AND course_rating <= 4.3;"""
explanation = run_queries(command)
for row in explanation:
    print(row[0])

Append  (cost=0.00..228.07 rows=360 width=77)
  ->  Seq Scan on good_rating_node_1 coursera_master_1  (cost=0.00..184.12 rows=357 width=77)
        Filter: ((course_rating >= '4.1'::double precision) AND (course_rating <= '4.3'::double precision))
  ->  Seq Scan on good_rating_node_2 coursera_master_2  (cost=0.00..14.05 rows=1 width=72)
        Filter: ((course_rating >= '4.1'::double precision) AND (course_rating <= '4.3'::double precision))
  ->  Seq Scan on good_rating_node_4 coursera_master_3  (cost=0.00..14.05 rows=1 width=72)
        Filter: ((course_rating >= '4.1'::double precision) AND (course_rating <= '4.3'::double precision))
  ->  Seq Scan on good_rating_node_8 coursera_master_4  (cost=0.00..14.05 rows=1 width=72)
        Filter: ((course_rating >= '4.1'::double precision) AND (course_rating <= '4.3'::double precision))


### Retrieving data based on duplication

The below record_id had a duplication factor of 3, which means the row had occurred 3 times in the original data. When data was stored in the database, duplicates were removed & size was the column kept to keep the record of it. The number is shown in the cell immediately below. There is only 1 row which has the record saved in the table with record_id 1178. 

However, the record of number of occurences is stored in the column size. When retrieving, based on the value mentioned in size, the number of rows will be returned correspondingly.



In [101]:
command = """SELECT * FROM coursera_master WHERE record_id=1178"""
data = run_queries(command)
print(len(data))
print(data[0])

1
('Essential Google Cloud Infrastructure: Core Services', 'Google Cloud', 'Conversant', 4.7, 'https://www.coursera.org/learn/gcp-infrastructure-core-services', 'This accelerated on-demand course introduces participants to the comprehensive and flexible infrastructure and platform services provided by Google Cloud with a focus on Compute Engine. Through a combination of video lectures, demos, and hands-on labs, participants explore and deploy solution elements, including infrastructure components such as networks, systems and applications services. This course also covers deploying practical solutions including customer-supplied encryption keys, security and access management, quotas and billing, and resource monitoring.  Prerequisites: To get the most out of this course, participants should have: � Completed Google Cloud Fundamentals: Core Infrastructure or have equivalent experience � Completed Essential Google Cloud Infrastructure: Foundation or have equivalent experience � Basic pr

In [99]:
command = """SELECT * FROM coursera_master c CROSS JOIN generate_series(1,c.size) as x WHERE record_id=1178"""
data = run_queries(command)
print(len(data))

3


### Replication

The implementation is done in a master-slave architecture with the replication factor of 1. All the operations done on the master will be mirrored in the slave so that there is always a backup available in case the master becomes unavailable. This ensures high availability of the system for access.

All the operations done above are performed on the master which is available at the port 5432. However, to show data replication, the slave is available on port 5433 & the query is run on the same server.

In [103]:
conn = psycopg2.connect(
            host="localhost",
            database="coursera",
            user="postgres",
            port=5433,
            password="rocky1405!")
cur = conn.cursor()
command = "select count(*) from coursera_master"
cur.execute(command)
rows = cur.fetchall()
print(rows[0][0])
cur.close()
conn.commit()
conn.close()

3424
